In [1]:
# Create GAN for independent and identically distributed data

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import secrets
from tqdm import tqdm
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import math
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

import keras
from keras import layers
from keras import ops
import gym
import d4rl


def create_data(
    num_samples=10000, initial_key1=None, initial_key2=None, ln=5, output_ln=5
):
    if initial_key1 is None:
        initial_key1 = np.random.randint(0, 2, 5)
        print("Initial key 1: ", initial_key1)
    if initial_key2 is None:
        initial_key2 = np.random.randint(0, 2, 5)
        print("Initial key 2: ", initial_key2)

    # data = initial_key
    key1 = initial_key1
    key2 = initial_key2
    x = []
    y = []
    # st = set()
    data = []
    for _ in tqdm(range(num_samples)):
        nxt1 = key1[0] ^ key1[1]
        nxt2 = key2[0] ^ key2[2]
        data.append(nxt1 ^ nxt2)
        if len(data) >= ln + output_ln:
            x.append(data[-(ln + output_ln) : -output_ln])
            # st.add(tuple(data[-10 * ln : -9 * ln]))
            y.append(data[-output_ln:])
        key1 = np.roll(key1, -1)
        key2 = np.roll(key2, -1)
        key1[-1] = nxt1
        key2[-1] = nxt2
    # print("Unique samples: ", len(st))
    # return np.array(x), np.array(y)
    x = np.array(x, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    x = x.reshape(x.shape[0], x.shape[1])
    y = y.reshape(y.shape[0], y.shape[1], 1)
    return np.array(y), np.array(x)


data_tmp = create_data(
    10000,
    ln=10,
    output_ln=512,
    initial_key1=np.array([1, 0, 1, 0, 1]),
    initial_key2=np.array([0, 0, 1, 1, 1]),
)

# data2 = create_data(
#     10000,
#     ln=10,
#     output_ln=512,
#     initial_key1=np.array([1, 0, 1, 0, 1]),
#     initial_key2=np.array([0, 0, 1, 1, 1]),
# )

2024-05-02 18:59:11.572395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 18:59:20.380277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/TensorRT-8.4.0.6//lib:/usr/local/cuda-11.6/lib64:/usr/local/apps/python-3.10.2/lib:/home2/shivam.sood/.mujoco/mujoco210/bin:/home2/shivam.sood/.mujoco/mujoco210/bin:/usr/local/apps/cuDNN/8.4.0-cuda-11.6/lib
2024-05-02 18:59:20.381459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [5]:
# data[0] = data[0].astype(np.float32)
# data[1] = data[1].astype(np.float32)
# data2[0] = data[0].astype(np.float32)
# data2[1] = data[1].astype(np.float32)
print(data_tmp[0].shape)
data = data_tmp[0][: 5000 - 10], data_tmp[1][: 5000 - 10] 
data2 = data_tmp[0][5000 - 10 : ], data_tmp[1][5000 - 10 : ]

# # data2 = data2.astype(np.float32)

(9479, 512, 1)


In [6]:
data = tf.data.Dataset.from_tensor_slices(data)
data2 = tf.data.Dataset.from_tensor_slices(data2)

In [7]:
# data
# data = tf.data.Dataset.from_tensor_slices(data)
# data2 = tf.data.Dataset.from_tensor_slices(data2)
num_epochs = 1  # train for at least 50 epochs for good results
image_size = 512


# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
widths = [32, 64, 96, 128]
has_attention = [False, False, True, True]
block_depth = 2

# optimization
batch_size = 64
ema = 0.999
learning_rate = 1e-3
weight_decay = 1e-4

In [8]:

def preprocess_image(data, labels):
    # center crop image
    # height = ops.shape(data["image"])[0]
    # width = ops.shape(data["image"])[1]
    # crop_size = ops.minimum(height, width)
    # image = tf.image.crop_to_bounding_box(
    #     data["image"],
    #     (height - crop_size) // 2,
    #     (width - crop_size) // 2,
    #     crop_size,
    #     crop_size,
    # )

    # resize and clip
    # for image downsampling it is important to turn on antialiasing
    # image = tf.image.grayscale_to_rgb(data["image"])
    # image = tf.image.resize(image, size=[image_size, image_size], antialias=True)
    return ops.clip(data, 0.0, 1.0), ops.clip(labels,0.0,1.0)


def prepare_dataset(dataset_name):
    # the validation dataset is shuffled as well, because data order matters
    # for the KID estimation
    return (
        dataset_name
        .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()
        .repeat(1)
        .shuffle(10 * batch_size)
        .batch(batch_size, drop_remainder=True)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )


# load dataset
train_dataset = prepare_dataset(data)
val_dataset = prepare_dataset(data)

In [9]:
@keras.saving.register_keras_serializable()
def sinusoidal_embedding(x):
    embedding_min_frequency = 1.0
    frequencies = ops.exp(
        ops.linspace(
            ops.log(embedding_min_frequency),
            ops.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = ops.cast(2.0 * math.pi * frequencies, "float32")
    embeddings = ops.concatenate(
        [ops.sin(angular_speeds * x), ops.cos(angular_speeds * x)], axis=2
    )
    return embeddings


def ResidualBlock(width):
    def apply(x, temb):
        input_width = x.shape[2]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv1D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)
        x = layers.Conv1D(width, kernel_size=3, padding="same", activation="swish")(x)
        temb = layers.Dense(width, activation="swish")(temb)
        x = layers.Add()([x, temb])
        x = layers.Conv1D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply


def DownBlock(i, width, block_depth):
    def apply(x, temb):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x, temb)
            skips.append(x)
        # x = layers.AveragePooling1D(pool_size=2)(x)
        x = layers.Conv1D(width, kernel_size=3, strides=2, padding="same")(x)
        return x

    return apply


def UpBlock(i, width, block_depth):
    def apply(x, temb):
        x, skips = x
        x = layers.UpSampling1D(size=2)(x)
        x = layers.Conv1D(width, kernel_size=3, padding="same")(x)
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x, temb)
        return x

    return apply


def get_network(image_size, widths, block_depth):
    noisy_images = keras.Input(shape=(image_size, 1))
    noise_variances = keras.Input(shape=(1, 1))
    labels = keras.Input(shape=(10,))
    # print("DASDA")
    e = layers.Lambda(sinusoidal_embedding, output_shape=(1, 32))(noise_variances)
    # l = layers.Embedding(2, 32)(labels)
    
    # print(e.shape,l.shape)
    temb = e
    # temb = keras.layers.Dense(32)(e)
    # temb = keras.layers.Dense(32)(temb)
    # l = keras.layers.Dense(32)(labels)
    l = keras.layers.Dense(32)(labels)
    l = keras.layers.Dense(32 * 4)(l)
    l = keras.layers.Dense(32, activation="sigmoid")(l)
    l = layers.Reshape((1,-1))(l)
    temb = keras.layers.Concatenate()([temb, l])
    e = layers.UpSampling1D(size=image_size)(temb)
    
    

    x = layers.Conv1D(widths[0], kernel_size=1)(noisy_images)
    x = layers.Concatenate()([x, e])

    # skips = []
    skips = [x]
    for i, width in enumerate(widths):
        x = DownBlock(i, width, block_depth)([x, skips], temb)
        


    for i in range(block_depth):
        x = ResidualBlock(width)(x, temb)
        # if i != block_depth - 1:
        #     x = layers.MultiHeadAttention(
        #         num_heads=8, key_dim=widths[-1] 
        #     )(x, x)

    for i, width in enumerate(reversed(widths)):
        x = UpBlock(i, width, block_depth)([x, skips], temb)
        
    x = layers.Concatenate()([x, skips.pop()])
    x = layers.Conv1D(1, kernel_size=1, kernel_initializer="zeros")(x)
    # x2 = layers.Conv1D(3, kernel_size=1, kernel_initializer="zeros", activation='softmax')(x)
    # x3 = layers.Conv1D(1, kernel_size=1, kernel_initializer="zeros", activation='sigmoid')(x)
    # x = layers.Concatenate()([x1,x2,x3])

    return keras.Model([noisy_images, noise_variances, labels], x, name="residual_unet")

In [10]:
@keras.saving.register_keras_serializable()
class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, block_depth):
        super().__init__()

        self.normalizer = layers.Normalization()
        self.network = get_network(image_size, widths, block_depth)
        self.ema_network = keras.models.clone_model(self.network)

    def compile(self, **kwargs):
        super().compile(**kwargs)

        self.noise_loss_tracker = keras.metrics.Mean(name="n_loss")
        self.image_loss_tracker = keras.metrics.Mean(name="i_loss")

    @property
    def metrics(self):
        return [self.noise_loss_tracker, self.image_loss_tracker]

    def denormalize(self, images):
        # convert the pixel values back to 0-1 range
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return ops.clip(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # diffusion times -> angles
        start_angle = ops.cast(ops.arccos(max_signal_rate), "float32")
        end_angle = ops.cast(ops.arccos(min_signal_rate), "float32")

        diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

        # angles -> signal and noise rates
        signal_rates = ops.cos(diffusion_angles)
        noise_rates = ops.sin(diffusion_angles)
        # note that their squared sum is always: sin^2(x) + cos^2(x) = 1

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, training, labels):
        # the exponential moving average weights are used at evaluation
        if training:
            network = self.network
        else:
            network = self.ema_network

        # predict noise component and calculate the image component using it
        # print(noisy_images.shape,noise_rates.shape, labels.shape)
        pred_noises = network([noisy_images, noise_rates**2, labels], training=training)
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps, labels):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # important line:
        # at the first sampling step, the "noisy image" is pure noise
        # but its signal rate is assumed to be nonzero (min_signal_rate)
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # separate the current noisy image to its components
            diffusion_times = ops.ones((num_images, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False, labels=labels
            )
            # network used in eval mode

            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step

        return pred_images

    def fill_reverse_diffusion(self, initial_image, initial_noise, diffusion_steps, labels):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps
        determined_len = initial_image.shape[0]
        fill_len = image_size - determined_len
        initial_images = tf.reshape(initial_image,(1, determined_len, 1))
        initial_images = tf.tile(initial_images, [num_images, 1, 1])
        initial_images = self.normalizer(initial_images, training=False)

        
        # important line:
        # at the first sampling step, the "noisy image" is pure noise
        # but its signal rate is assumed to be nonzero (min_signal_rate)
        next_noisy_images = initial_noise
        for step in range(diffusion_steps+1):
            noisy_images = next_noisy_images
            # separate the current noisy image to its components
            diffusion_times = ops.ones((num_images, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False, labels=labels
            )
            # network used in eval mode
            
            
            # print(initial_images.shape, pred_images[:, determined_len:].shape, determined_len)
            pred_images = tf.concat([initial_images, pred_images[:, determined_len:]], axis=1)

            # pred_noises = tf.concat([tf.zeros((num_images,determined_len,6)), pred_noises[:, determined_len:]], axis=1)
            # print(pred_images.shape, image_size)
            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step
        # print(initial_images)
        return pred_images

    def generate_fill(self, num_images, diffusion_steps, labels, initial_image):
        # noise -> images -> denormalized images
        initial_noise = keras.random.normal(shape=(num_images, image_size, 1))
        generated_images = self.fill_reverse_diffusion(
            initial_image, initial_noise, diffusion_steps, labels
        )
        generated_images = self.denormalize(generated_images)
        return generated_images
    def generate(self, num_images, diffusion_steps, labels):
        # noise -> images -> denormalized images
        initial_noise = keras.random.normal(shape=(num_images, image_size, 1))
        generated_images = self.reverse_diffusion(
            initial_noise, diffusion_steps, labels
        )
        generated_images = self.denormalize(generated_images)
        return generated_images

    def train_step(self, images):
        # print(images[0].shape,images[1].shape)
        # normalize images to have standard deviation of 1, like the noises
        images, labels = images
        # print(images.shape)
        images = self.normalizer(images, training=True)
        # print(images.shape)
        noises = keras.random.normal(shape=(batch_size, image_size, 1))

        # sample uniform random diffusion times
        diffusion_times = keras.random.uniform(
            shape=(batch_size, 1, 1), minval=0.0, maxval=1.0
        )
        # print(images.shape)
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        # print(images.shape,signal_rates.shape)
        noisy_images = signal_rates * images + noise_rates * noises
        # print(noisy_images.shape)
        with tf.GradientTape() as tape:
            # train the network to separate noisy images to their components
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True, labels=labels
            )

            noise_loss = self.loss(noises, pred_noises)  # used for training
            image_loss = self.loss(images, pred_images)  # only used as metric

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    def test_step(self, images):
        images, labels = images
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=False)
        noises = keras.random.normal(shape=(batch_size, image_size, 1))

        # sample uniform random diffusion times
        diffusion_times = keras.random.uniform(
            shape=(batch_size, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        # use the network to separate noisy images to their components
        pred_noises, pred_images = self.denoise(
            noisy_images, noise_rates, signal_rates, training=False, labels=labels
        )

        noise_loss = self.loss(noises, pred_noises)
        image_loss = self.loss(images, pred_images)

        self.image_loss_tracker.update_state(image_loss)
        self.noise_loss_tracker.update_state(noise_loss)

        # measure KID between real and generated images
        # this is computationally demanding, kid_diffusion_steps has to be small
        # images = self.denormalize(images)
        # generated_images = self.generate(
        #     num_images=batch_size, diffusion_steps=kid_diffusion_steps
        # )
        # self.kid.update_state(images, generated_images)

        return {m.name: m.result() for m in self.metrics}

    def plot_images(self, epoch=None, logs=None, num_rows=1, num_cols=1):
        # plot random generated images for visual evaluation of generation quality
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=20,
        )

        print(generated_images)

In [14]:
# create and compile the model
model = DiffusionModel(image_size, widths, block_depth)
# below tensorflow 2.9:
# pip install tensorflow_addons
# import tensorflow_addons as tfa
# optimizer=tfa.optimizers.AdamW
opt = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    # optimizer=opt,
    loss=keras.losses.mean_absolute_error,
)
# pixelwise mean absolute error is used as loss

# save the best model based on the validation KID metric
# checkpoint_path = "checkpoints/diffusion_model.weights.h5"
# checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path,
#     save_weights_only=True,
#     monitor="val_i_loss",
#     # mode="min",
#     save_best_only=True,
# )

# calculate mean and variance of training dataset for normalization
# images = np.array([x for x, y in train_dataset])
# print(images.shape)
model.normalizer.adapt(train_dataset.map(lambda x, y: x))
# print(np.array([y for x, y in train_dataset]).shape)
# run training and plot generated images periodically
# model.fit(
#     train_dataset,
#     epochs=5,
#     validation_data=val_dataset,
#     callbacks=[
#         # keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
#         # checkpoint_callback,
#     ],
# )

In [15]:
model.load_weights('scipher.weights.h5')

/home2/shivam.sood/shivam_env/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 318 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [16]:

model.fit(
    train_dataset,
    epochs=500,
    validation_data=val_dataset,
    callbacks=[
        # keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        # checkpoint_callback,
    ],
)

Epoch 1/500


2024-05-02 19:07:50.444367: I tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:325] ptxas warning : Registers are spilled to local memory in function 'fusion_1465', 18312 bytes spill stores, 28116 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'fusion_1290', 8 bytes spill stores, 8 bytes spill loads



77/77 ━━━━━━━━━━━━━━━━━━━━ 57s 141ms/step - n_loss: 0.2424 - i_loss: 0.4923 - val_i_loss: 1.0246 - val_n_loss: 0.4411
Epoch 2/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2333 - i_loss: 0.4772 - val_i_loss: 0.9718 - val_n_loss: 0.4309
Epoch 3/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2322 - i_loss: 0.4848 - val_i_loss: 0.9695 - val_n_loss: 0.4178
Epoch 4/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2350 - i_loss: 0.4765 - val_i_loss: 0.9012 - val_n_loss: 0.4107
Epoch 5/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2359 - i_loss: 0.4814 - val_i_loss: 0.9141 - val_n_loss: 0.3972
Epoch 6/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2324 - i_loss: 0.4909 - val_i_loss: 0.8433 - val_n_loss: 0.3913
Epoch 7/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - n_loss: 0.2345 - i_loss: 0.4800 - val_i_loss: 0.8282 - val_n_loss: 0.3809
Epoch 8/500
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - n_loss: 0.2304 - i_loss: 0.4865 - val_i_loss: 0.8163 - val_n


KeyboardInterrupt



In [17]:
model.save_weights("scipher.weights.h5")

In [18]:
# test_data_img, test_data_label = create_data(
#     600,
#     ln=10,
#     output_ln=512,
#     initial_key1=np.array([1, 0, 1, 0, 1]),
#     initial_key2=np.array([0, 0, 1, 1, 1]),
# )
print(data_tmp[0][5000 - 10: 5000 ].shape)
total_acc = 0
for a, b in tqdm(zip(data_tmp[0][5000 - 10: 5000 ], data_tmp[1][5000 - 10: 5000 ])):
    x = (model.generate(1,20, tf.convert_to_tensor([b])))
    x = x[0]
    x = x.numpy()
    x = np.rint(x)
    acc = np.sum(x == a) / 512 
    total_acc += acc
# total_acc /= 1000

(10, 512, 1)


10it [00:33,  3.32s/it]


In [19]:
# x_fixed = datagen.denormalize_values(x)
print(total_acc /10)
# print(x_fixed[0])

0.4998046875


In [22]:
model.network.summary()

Model: "residual_unet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        160 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      4,224 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 32)     │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 32)     │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 32, 7)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1, 64)     │          0 │ lambda[0][0],     │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 32, 64)    │        512 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d       │ (None, 32, 64)    │          0 │ concatenate[0][0] │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32, 128)   │          0 │ conv1d[0][0],     │
│ (Concatenate)       │                   │            │ up_sampling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 128)   │        256 │ concatenate_1[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 32, 64)    │     24,640 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1, 64)     │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 64)    │          0 │ conv1d_2[0][0],   │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 32, 64)    │     12,352 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 32, 64)    │      8,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 32, 64)    │          0 │ conv1d_3[0][0],   │
│                     │                   │            │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64)    │        128 │ add_1[0][0]       │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 17,360,455 (66.22 MB)

 Trainable params: 17,347,655 (66.18 MB)

 Non-trainable params: 12,800 (50.00 KB)